# Simulacion de Trayectoria Ascendente y Descendete de un Globo-sonda

## Tabla de Contenido
* [1. Preparación del Notebook](#prework)
* [2. Descripición de la Misión ](#mision)
    * [2.1 Propiedades del Sistema](#propiedades)
    * [2.2 Estado del Sistema](#estado)
    * [2.3 Perfil de la misión](#perfil)
* [3. Modelo del Ambiente](#ambiente)
    * [3.1 Atmosférico](#atmosferico)
    * [3.2 Termico](#termico)
    * [3.3 Tierra](#tierra)
* [4. Modelo de Trayectoria](#trayectoria)
    * [4.1 Viento](#atmosferico)
    * [4.2 Geométrico](#termico)
    * [4.3 Dinámico](#tierra)


<!---

* [5. Resultados y Discusión](#trayectoria)
    * [5.1 Graficas](#atmosferico)
    * [5.2 Metricas](#termico)
    * [5.3 No sé jeje](#tierra)

-->

> 📝 **Nota:** Todo lo presentado acá aún se encuentra en desarrollo y muchas cosas podran cambiar a medida se avanza en ello.

# 1. Preparación del Notebook  <a name="prework"></a>

Se cargan las librerias o módulos necesarios para la simulación.

In [6]:
##matplotlib inline      
import math 
import matplotlib.pyplot as plt   # Generación de gráficos a partir de listas o arrays. 
import numpy as np                # Manipulación de vectores, matrix y tensores
import pandas as pd

# 2. Descripición de la Misión  <a name="mision"></a>

<figure>
    <img src = "../img/FasesDeVueloMission_StratoBalloon.png"
         alt = "Fase de Vuelo"
         name = "Fase de Vuelo"
         style="height 400px; width800px;">
    <figcaption> Fig. 1 Fase de Vuelo.</figcaption>
</figure>




Las operaciones con globos de gran altitud se resumen en 5 fases
- **Fase 1** es el lanzamiento de la sonda desde unas coordenadas predefinidas.
- **Fase 2** es el ascenso de la sonda hasta una altura de varios kilómetros determinado por el fabricante del globo [7] y el peso de los instrumentos.
- **Fase 3** es la explosión del globo
- **Fase 4** es el descenso con solo el paracaídas y la carga útil.
- **Fase 5** que es la recuperación de la sonda con ayuda de los instrumentos de navegación

## 2.1 Propiedades del Sistema  <a name="propiedades"></a>


Son las cosas que son costantes de la misión por asi decirlo

In [ ]:
# Sonde    
BalloonMass = 0.5       # kg
PayloadMass = 2.1       # kg
TetherMass = 0.1        # kg
ParachuteMass = 0.5     # kg
rho_He0 = 0.179         # Density of Helium at sea level (kg/m3)
C_dBalloon = 0.47       # drag coefficient of balloon
C_dadded = 0.25         # c_Added coefficient

# Parachute
Cd_pchute = 0.3	# Parachute drag coefficient
Cd_spchute = 0.1	# parachute side drag coefficient
PDiam = 2		# parachute diameter (m)
PSideL = 0.1		# parachute side length, flaps (m)

## 2.2 Estado del Sistema  <a name="estado"></a>

Fases de vuelo y las condiciones en las cuales se encuentra la sonda en el espacio

In [ ]:
system_stage = 0	# Mission stage  0=balloon 1=parachute 2=gliding
#   ["balloon", "free_falling", "parachute"]	
v_init = 0			# velocity m/s
x_init = 0			# x,y,z position
y_init = 0
# Seria bueno tener la altura del lugar
z_init = 1
balloon_diam_init = 2.3	# balloon initial diamenter


#           LANZAMIENTO
#   Autodromo Internacional El Jabalí
#   https//goo.gl/maps/sARALqQY2UCwtPMZ7
lat_init = 13.808826
lon_init = -89.328988

## 2.3 Perfil de la mision  <a name="perfil"></a>


Objetivos de activación de la misión

In [ ]:
balloon_separation = 30e3	# m
parachute1_separation = 16000	# m

system_state = [system_stage, v_init, x_init, y_init, z_init, balloon_diam_init, gamma_init, chi_init, lat_init, lon_init]
system_state_hist = system_state.copy()
wvector_hist = []
glider_control = [0, 0]



In [ ]:
while (system_state[4] > 0):

    wvector = windvector(0, 0, system_state[4], 0)
    wvector_hist.append(wvector)

    x0 = system_state[2]
    y0 = system_state[3]
    z0 = system_state[4]

    if (system_state[0] == 0) and (z0 > balloon_separation):
        system_state[1] = 0
        system_state[6] = -np.pi/2
        system_state[0] = 1  # go to parachute
    elif (system_state[0] == 1) and (z0 < parachute1_separation):
        system_state[6] = -np.pi/2
        system_state[0] = 2  # go to gliding

    z_alt = system_state[4]

    if (system_state[0] == 0):
        balloon_properties = [BalloonMass, PayloadMass, TetherMass, ParachuteMass, rho_He0, C_dBalloon, C_dadded]

        balloon_state = system_state[1:8]
        balloon_state = next_balloon_state(balloon_properties, balloon_state, wvector)
        system_state = [system_state[0], *balloon_state, system_state[8], system_state[9]]

    elif (system_state[0] == 1):
        parachute_properties = [PayloadMass, ParachuteMass, Cd_pchute, PDiam, PSideL, Cd_spchute]

        parachute_state = [*system_state[1:5], *system_state[6:8]]
        parachute_state = next_parachute_state(parachute_properties, parachute_state, wvector)
        system_state = [system_state[0], *parachute_state[:4], 0, *parachute_state[4:], system_state[8], system_state[9]]

    elif (system_state[0] == 2):
        glider_properties = [PayloadMass, Wing_Area]
        glider_state = [abs(system_state[1]), system_state[2:5], system_state[6:8]]
        
        glider_control = [0, 0]

        
        glider_state = next_glider_state_wind(glider_properties, glider_state, glider_control, wvector)
        system_state = [system_state[0], *glider_state[:4], 0, *glider_state[4:], system_state[8], system_state[9]]
    
    dx = x0 - system_state[2]
    dy = y0 - system_state[3]
    dz = z0 - system_state[4]

    nLat, nLon, nH, nAz = flatdisp2geo(system_state[9], system_state[10], system_state[4], -dx, -dy, -dz)
    system_state[9] = nLat
    system_state[10] = nLon

    system_state_hist.append(system_state)




V_data = system_state_hist[:,1]
X_data = system_state_hist[:,2]
Y_data = system_state_hist[:,3]
Z_data = system_state_hist[:,4]

Gamma_data = system_state_hist[:,6]
Chi_data = system_state_hist[:,7]

Vx_data = V_data * np.cos(Chi_data) * np.cos(Gamma_data)
Vy_data = V_data * np.sin(Chi_data) * np.cos(Gamma_data)

Lat_data = system_state_hist[:,8]
Lon_data = system_state_hist[:,9]


# 3. Modelo del Ambiente  <a name="ambiente"></a>




## 3.1 Atmosférico  <a name="atmosferico"></a>


Se utiliza el modelo atmosférico estándar, en inglés The International Standard Atmosphere (ISA), para calcular la temperatura, la presión y densidad del atmósfera. **En el model atmosférico todas las funciones generadas son función de la altitud.**

Como el objetivo de la misión es llegar a los 30 Km, el modelo se divide en tres regiones diferentes:

- **Primera región (z ≤ 11,019m):** se asume que la temperatura disminuye linealmente con la altitud a una tasa.  Existe variación de la presion y la densidad atmosférica. Se configura las variables iniciales de temperatura, presión y densidad a ttt.tt, ppp.pp y dddd.dd respectivamente. 

- **Segunda región (11,019m < z ≤  20,063m):** la capa es tipo isotermica por lo que se asume que la temperatura es constante a 216.65 K. Existe variación de la presion y la densidad atmosférica. Se configura las variables iniciales de temperatura, presión y densidad a ttt.tt, ppp.pp y dddd.dd respectivamente.

- **Tercera región (z ≤ 30,000m):** se asume que la temperatura aumenta linealmente con la altitud a una tasa x. Existe variación de la presion y la densidad atmosférica. Se configura las variables iniciales de temperatura, presión y densidad a ttt.tt, ppp.pp y dddd.dd respectivamente.

La información se resume en la siguiente tabla:

** se crea una tabla resumen acá **

In [ ]:

T0 =0;lambda0 =0; P0 =0; z0 =0;
r =0;




In [4]:
import numpy as np

def eath_gravity(z):
    g0 = 9.80665
    Re = 6.371e6
    g = g0 * (Re / (Re + z))**2

    return g

T0, lambda0, P0, z0 = 0, 0, 0, 0

def atm_tmp(z):
    r = np.zeros_like(z)

    for i, z_val in np.ndenumerate(z):
        if z_val < 11019:
            z0 = 0
            T0 = 288.15
            lambda0 = -0.0065
            P0 = 101325.0

            r[i] = T0 + lambda0 * (z_val - z0)

        elif z_val < 20063:
            z0 = 11019
            T0 = 216.65

            r[i] = T0

        elif z_val < 32162:
            z0 = 20063
            T0 = 216.65
            lambda0 = 0.0010
            P0 = 5474.89

            r[i] = T0 + lambda0 * (z_val - z0)

    return r

def atm_pressure(z):
    r = np.zeros_like(z)

    for i, z_val in np.ndenumerate(z):
        if z_val < 11019:
            z0 = 0
            T0 = 288.15
            lambda0 = -0.0065
            P0 = 101325.0

            r[i] = P0 * ((T0 / atm_tmp(z)) ** (earth_gravity(z) * 0.0289644 / (8.31432 * lambda0)))


        elif z_val < 20063:
            z0 = 11019
            T0 = 216.65
            P0 = 22632.10

            r[i] = P0 * np.exp(-earth_gravity(z) * 0.0289644 * (z - z0) / (8.31432 * atm_tmp(z)))

        elif z_val < 32162:
            z0 = 20063
            T0 = 216.65
            lambda0 = 0.0010
            P0 = 5474.89
            

            r[i] = r = P0 * ((T0 / atm_tmp(z)) ** (earth_gravity(z) * 0.0289644 / (8.31432 * lambda0)))

    return r

def atm_rho(z):
    r = 0
    
    if z < 11019:
        z0 = 0
        T0 = 288.15
        lambda0 = -0.0065
        P0 = 101325.0

    elif z < 20063:
        z0 = 11019
        T0 = 216.65
        P0 = 22632.10
        
    elif z < 32162:
        z0 = 20063
        T0 = 216.65
        lambda0 = 0.0010
        P0 = 5474.89
    
    r = atm_pressure(z) / (atm_tmp(z) * 287.04)
    
    return r


In [ ]:
import numpy as np

def eath_gravity(z):
    g0 = 9.80665
    Re = 6.371e6
    g = g0 * (Re / (Re + z))**2

    return g

def atm_tmp(z):
    T0, lambda0, P0, z0 = 0, 0, 0, 0
    r = np.zeros_like(z)
    
    mask1 = z < 11019
    mask2 = np.logical_and(z >= 11019, z < 20063)
    mask3 = np.logical_and(z >= 20063, z < 32162)
    
    z0[mask1] = 0
    z0[mask2] = 11019
    z0[mask3] = 20063
    
    T0[mask1] = 288.15
    T0[mask2] = 216.65
    T0[mask3] = 216.65
    
    lambda0[mask1] = -0.0065
    lambda0[mask2] = 0
    lambda0[mask3] = 0.0010
    
    P0[mask1] = 101325.0
    P0[mask2] = 22632.10
    P0[mask3] = 5474.89
    
    r[mask1] = T0[mask1] + lambda0[mask1]*(z[mask1]-z0[mask1])
    r[mask2] = T0[mask2]
    r[mask3] = T0[mask3] + lambda0[mask3]*(z[mask3]-z0[mask3])
    
    return r





In [ ]:
h = np.linspace(0, 30e3, 1000)


# variables ndarray
altura = h
temperatura = atm_tmp(h)
presion = atm_pressure(h)
densidad = atm_rho(h)
gravedad =  eath_gravity(h)

# creación del DataFrame de Pandas con las variables
df = pd.DataFrame({
    'height[m]': altura, 
    'temperature[K]': temperatura,
    'pressure[Pa]': presion,
    'density[Kg/m³]': densidad,
    'gravity[m/s²]': gravedad,
    
    # LO SIGUIENTE SON FUTURAS VARIABLES E IDEAS:
    # 'time/date[]': time/date,
    # 'latitude[]': latitud, 
    # 'longitude[]': longitud,
    # 'wind x[]': viento x,
    # 'wind y[]': viento y,
    # 'vel x[]': vel x,
    # 'vel y[]': vel y,
    # 'status[]': estado
    })

# guardar el DataFrame del conjunto de ndarray como archivo .csv
df.to_csv('../data/parameters.csv', index=False)



En status pues se tendrian todas las fases de la mision, como ejemplo estas ideas:
- ground
- ascendenting
- descendenting



(Si el data frame crece mucho puedo hacer esto)

Cuando tenga muchas variables y desee automatizar el guardado de las variables:


In [ ]:
# h = np.linspace(0, 30e3, 1000)

# # variables ndarray
# altura = h
# variable1 = atm_tmp(h)
# variable2 = atm_pressure(h)
# variable3 = atm_rho(h)
# variable4 = eath_gravity(h)
# variable5 = h 
# variable6 = h
# variable7 = h
# variable8 = h
# variable9 = h
# variable10 = h

# # para automatiza creo un diccionario vacío para almacenar las variables
# data = {}

# # agrego las variables al diccionario utilizando un bucle
# for i in range(1, 11):
#     variable_name = 'Variable ' + str(i)
#     variable = eval('variable' + str(i))
#     data[variable_name] = variable

# # crear un DataFrame de Pandas con las variables
# df = pd.DataFrame(data)

# # guardar el DataFrame del conjunto de ndarray como archivo .csv
# df.to_csv('../data/parameters.csv', index=False)


LO MALO DE LO ANTERIOR:

Automatiza si, pero lo hace guardando todo con Variable X y X lo sustituye por la iteración, me gustaria que tome el nombre de la variable de la función respectiva, luego vere como soluciono eso.

## 3.2 Térmico

## 3.3 Tierra  <a name="tierra"></a>

### 3.3.1 Gravedad

Se usa una aproximación para la gravedad estándar y puede existir variaciones en la gravedad real en función de la ubicación. A continuación el code:

In [3]:
def earth_gravity(z):
    g0 = 9.80665
    Re = 6371000
    g = g0 * ((Re / (Re + z)) ** 2)
    return g


# 4. Modelo de Trayectoria

## Parachute



Saber las dimensiones del parachute
Peso para detener la carga

¿Cómo deberminar la resistencia que debe ofrecer el paracaidas a la carga?


**Formas de hacer los cálculos**

**Vertical** 
zo = altura a la que se lanza


**horixontal**
xo =  posición horizontal en al que se encuentra


Aplicar las ecuaciones de Cinemática

# Información de sesión

Como una buena práctica colocamos la información de sesión al final de este notebook realizado. De esta forma, otras personas podrán ver qué versiones de librerías y dependencias se utilizaron para nuestro análisis.

Además, esto permite ver si llegara alguien a tener un problema, puede compartir esta información con la comunidad o nosotros los desarrolladores de este notebook para que sea más fácil replicar el error y poder ayudar.

In [ ]:
# Importamos la librería
import session_info

# Lanzamos el comando
session_info.show()
